In [ ]:
from __future__ import division
from collections import OrderedDict
import time

from quantopian.algorithm import order_optimal_portfolio
from quantopian.algorithm import attach_pipeline, pipeline_output, order_optimal_portfolio
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS
import quantopian.optimize as opt
from quantopian.optimize import TargetWeights
import quantopian.algorithm as algo
from quantopian.pipeline.experimental import risk_loading_pipeline

import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import Imputer, StandardScaler

In [ ]:
holding_days = 10 
days_for_analysis = 45

## Define the machine learnign model.
class Predictor(CustomFactor):
    
 
    # Factors that use to predict 
    factor_dict = OrderedDict([
              ('Open Price' , USEquityPricing.open),
              ('Volume' , USEquityPricing.volume),
              ('cf_yield' , Fundamentals.cf_yield),
              ('earning_yield' , Fundamentals.earning_yield),
              ('pb_ratio' , Fundamentals.pb_ratio),
              ('pe_ratio' , Fundamentals.pe_ratio), 
              ('roa' , Fundamentals.roa)
    ])
 
    columns = factor_dict.keys()
    inputs = factor_dict.values()
 
    
    def compute(self, today, assets, out, *inputs):
        
        
        ## Import Data and define y.
        
        inputs = OrderedDict([(self.columns[i] , pd.DataFrame(inputs[i]).fillna(method='ffill',axis=1).fillna(method='bfill',axis=1)) for i in range(len(inputs))]) # bring in data with some null handling.
        num_secs = len(inputs['Open Price'].columns)
        y = (np.log(inputs['Open Price']) - np.log(inputs['Open Price'].shift(holding_days))).shift(-holding_days-1).dropna(axis=0,how='all').stack(dropna=False)
        
        ## Get rid of our y value as an input into our machine learning algorithm.
        del inputs['Open Price']
 
        ## Munge X and y
        x = pd.concat([df.stack(dropna=False) for df in inputs.values()], axis=1)
        x = Imputer(strategy='median',axis=1).fit_transform(x) # fill nulls.
        y = np.ravel(Imputer(strategy='median',axis=1).fit_transform(y)) # fill nulls.
        scaler = StandardScaler()
        x = scaler.fit_transform(x) # demean and normalize
 
        ## Run Model
        model = LinearRegression()
        model_x = x[:-num_secs*(holding_days+1),:]
        model.fit(model_x, y)
 
        out[:] = model.predict(x[-num_secs:,:])

In [ ]:
def initialize(context):
    
    # Rebalance every day, 1 hour after market open.
    algo.schedule_function(
        rebalance,
        algo.date_rules.every_day(),
        algo.time_rules.market_open(hours=1),
    )

    # contrains to the contest
    context.max_posTam = 0.025
    context.max_lever = 0.98
    context.max_Volum = 0.85

    algo.attach_pipeline(risk_loading_pipeline(), 'risk_loading_pipeline')
    algo.attach_pipeline(make_pipeline(), 'pipeline')

In [ ]:
def make_pipeline():
   
    # Base universe set to the QTradableStocksUS
    base_universe = QTradableStocksUS()
    
    # use the model
    pipe = Pipeline(
        columns={
           'Model': Predictor(
                window_length=days_for_analysis
                ,mask=base_universe
            )
        }
        ,screen = base_universe
    )
    return pipe

In [ ]:
def before_trading_start(context, data):
    
    # Obtain the Pipeline output and save this in the context
    context.output = pipeline_output('pipeline')
    context.risk_factor_betas = pipeline_output('risk_loading_pipeline')

In [ ]:
def rebalance(context, data):
    
    pipeline_output_df = pipeline_output('pipeline').dropna(how='any')
    
    # Constrains
    constrain_posTam = opt.PositionConcentration.with_equal_bounds(-1.5*context.max_posTam,context.max_posTam)
    max_lever = opt.MaxGrossExposure(context.max_lever)
    max_Volum = opt.MaxTurnover(context.max_Volum) 
    dollar_net = opt.DollarNeutral()
    
    # Add for machine Learning
    todays_predictions = pipeline_output_df.Model
    target_weight_series = todays_predictions.sub(todays_predictions.mean())
    
    target_weight_series = 3.2*target_weight_series 
    
    #Risk Model Exposure
    factor_risk_constraints = opt.experimental.RiskModelExposure(
        context.risk_factor_betas,
        version=opt.Newest
    )

    order_optimal_portfolio(
        objective=TargetWeights(target_weight_series),
        constraints=[
            constrain_posTam,
            max_lever,
            max_Volum,
            factor_risk_constraints,
            dollar_net
            
        ]
    )
    pass